In [40]:
import pandas as pd
import unicodedata
import re
import duckdb
from io import StringIO
from google.cloud import storage
import wget

def remove_accents(text):
    """
    Remove accent marks from input text while preserving the base characters.
    Also handles special characters like Đ/đ.
    
    Example:
    "Nikola Đurišić" -> "Nikola Durisic"
    """
    # First, handle special characters that need specific replacements
    special_chars = {
        'Đ': 'D', 'đ': 'd',  # Serbian/Croatian D with stroke
        'Ł': 'L', 'ł': 'l',  # Polish L with stroke
        'Ø': 'O', 'ø': 'o',  # Danish/Norwegian O with stroke
        'Ŧ': 'T', 'ŧ': 't',  # Sami T with stroke
        'Æ': 'AE', 'æ': 'ae',  # Æ/æ ligature
        'Œ': 'OE', 'œ': 'oe',  # Œ/œ ligature
        'ß': 'ss',  # German eszett
    }
    
    for char, replacement in special_chars.items():
        text = text.replace(char, replacement)
    
    # Normalize the text to decompose characters into base character and accent mark
    normalized_text = unicodedata.normalize('NFKD', text)
    
    # Filter out the non-spacing marks (accent marks)
    result = ''.join(c for c in normalized_text if not unicodedata.category(c).startswith('Mn'))
    
    return result

# URL of the CSV file
filename = 'playeroftheweek.csv'
url = f'https://storage.googleapis.com/nba_award_predictor/nba_data/{filename}'
wget.download(url)
# Read in the playeroftheweek csv
playeroftheweek_df = pd.read_csv(filename)

# Clean each player's full name
playeroftheweek_df["player"] = playeroftheweek_df["player"].apply(remove_accents)

# Bring in name mapping table for names to help match all names to the format seen in the NBA API
filename = 'name_mappings.csv'
url = f'https://storage.googleapis.com/nba_award_predictor/nba_data/{filename}'
wget.download(url)
# Read in the name_mappings csv
name_mapping_df = pd.read_csv(filename)

# Bring in nba player lookup table to map the cleaned names to player IDs. Same player IDs from the NBA API.
filename = 'nba_player_lookup.csv'
url = f'https://storage.googleapis.com/nba_award_predictor/nba_data/{filename}'
wget.download(url)
# Read in the nba_player_lookup csv
nba_player_lookup_df = pd.read_csv(filename)

# Clean each player's full name
nba_player_lookup_df["player_name"] = nba_player_lookup_df["player_name"].apply(remove_accents)

query = """
SELECT * FROM playeroftheweek_df

"""

player_of_the_week_df = duckdb.query(query).df()#.drop(['person_id', 'player', 'display_last_comma_first', 'display_fi_last', 'player_slug', 'in_table_name', 'nba_lookup_name'], axis=1)
player_of_the_week_df

# #Rearrange columns
# cols = common_player_info_df.columns.tolist()
# new_cols = [cols[-1], cols[-2]] + cols[:-2]
# common_player_info_df = common_player_info_df[new_cols]
# common_player_info_df.to_csv('common-player-info.csv')


# # Path to your credentials file
# credentials_path = 'cis-5450-final-project-485661e2f371.json'

# # Set up the client with your credentials
# storage_client = storage.Client.from_service_account_json(credentials_path)

# # Specify your bucket name
# bucket_name = 'nba_award_predictor'
# bucket = storage_client.bucket(bucket_name)

# # Define blob (file in GCS) and upload from the local file
# blob = bucket.blob('nba_data/common-player-info.csv')
# blob.cache_control = "max-age=0"
# blob.upload_from_filename('common-player-info.csv')

# print(f"File uploaded to gs://{bucket_name}/nba_data/common-player-info.csv")


,season,player,conference,date,team,pos,height,weight,age,Pre-Draft Team,Draft Yr,yos
0,2025-2026,Tyrese Maxey,East,2025-11-03,Philadelphia Sixers,PG,6-2,200,25,Kentucky,2020,5
1,2025-2026,Shai Gilgeous-Alexander,West,2025-11-03,Oklahoma City Thunder,PG,6-6,195,27,Kentucky,2018,7
2,2025-2026,Victor Wembanyama,West,2025-10-27,San Antonio Spurs,F,7-4,235,22,Boulogne-Levallois (France),2023,2
3,2025-2026,Giannis Antetokounmpo,East,2025-10-27,Milwaukee Bucks,F,6-11,243,31,Filathlitikos Div II Greece (Greece),2013,12
4,2024-2025,James Harden,West,2025-04-14,Los Angeles Clippers,SG,6-5,220,35,Arizona State,2009,15
...,...,...,...,...,...,...,...,...,...,...,...,...
1559,1979-1980,Phil Ford,None,1979-11-18,Kansas City Kings,G,6-2,175,24,North Carolina,1978,1
1560,1979-1980,Magic Johnson,None,1979-11-11,Los Angeles Lakers,PG,6-9,255,20,Michigan State,1979,0
1561,1979-1980,Marques Johnson,None,1979-11-04,Milwaukee Bucks,GF,6-7,218,24,UCLA,1977,2
1562,1979-1980,Micheal Ray Richardson,None,1979-10-28,New York Knicks,PG,6-5,189,24,Montana,1978,1


In [57]:
import pandas as pd
import unicodedata
import re
import duckdb
from io import StringIO
from google.cloud import storage
import wget

def remove_accents(text):
    """
    Remove accent marks from input text while preserving the base characters.
    Also handles special characters like Đ/đ.
    
    Example:
    "Nikola Đurišić" -> "Nikola Durisic"
    """
    # First, handle special characters that need specific replacements
    special_chars = {
        'Đ': 'D', 'đ': 'd',  # Serbian/Croatian D with stroke
        'Ł': 'L', 'ł': 'l',  # Polish L with stroke
        'Ø': 'O', 'ø': 'o',  # Danish/Norwegian O with stroke
        'Ŧ': 'T', 'ŧ': 't',  # Sami T with stroke
        'Æ': 'AE', 'æ': 'ae',  # Æ/æ ligature
        'Œ': 'OE', 'œ': 'oe',  # Œ/œ ligature
        'ß': 'ss',  # German eszett
    }
    
    for char, replacement in special_chars.items():
        text = text.replace(char, replacement)
    
    # Normalize the text to decompose characters into base character and accent mark
    normalized_text = unicodedata.normalize('NFKD', text)
    
    # Filter out the non-spacing marks (accent marks)
    result = ''.join(c for c in normalized_text if not unicodedata.category(c).startswith('Mn'))
    
    return result

# URL of the CSV file
filename = 'playeroftheweek.csv'
url = f'https://storage.googleapis.com/nba_award_predictor/nba_data/{filename}'
wget.download(url)
# Read in the playeroftheweek csv
playeroftheweek_df = pd.read_csv(filename)

# Clean each player's full name
playeroftheweek_df["player"] = playeroftheweek_df["player"].apply(remove_accents)

# Bring in name mapping table for names to help match all names to the format seen in the NBA API
filename = 'name_mappings.csv'
url = f'https://storage.googleapis.com/nba_award_predictor/nba_data/{filename}'
wget.download(url)
# Read in the name_mappings csv
name_mapping_df = pd.read_csv(filename)

# Bring in nba player lookup table to map the cleaned names to player IDs. Same player IDs from the NBA API.
filename = 'nba_player_lookup.csv'
url = f'https://storage.googleapis.com/nba_award_predictor/nba_data/{filename}'
wget.download(url)
# Read in the nba_player_lookup csv
nba_player_lookup_df = pd.read_csv(filename)

# Clean each player's full name
nba_player_lookup_df["player_name"] = nba_player_lookup_df["player_name"].apply(remove_accents)

query = """
WITH CTE AS (
SELECT * FROM playeroftheweek_df
LEFT JOIN name_mapping_df
ON playeroftheweek_df.player = name_mapping_df.in_table_name
)
,CTE2 AS (
SELECT *,
CASE WHEN nba_lookup_name IS NULL THEN player
ELSE nba_lookup_name
END AS player_full_name
FROM CTE
)

SELECT CTE2.*
,nba_player_lookup_df.player_id AS true_player_id
FROM CTE2
JOIN nba_player_lookup_df
ON CTE2.player_full_name = nba_player_lookup_df.player_name
WHERE 1=1
AND nba_player_lookup_df.player_id NOT IN (76616, 120, 698, 7714) -- Taking out players who share the same name with someone else because it messes up the join

"""

player_of_the_week_df = duckdb.query(query).df().drop(['in_table_name', 'nba_lookup_name', 'player_id', 'Unnamed: 3', 'player_full_name'], axis=1)
player_of_the_week_df

#Rearrange columns
cols = player_of_the_week_df.columns.tolist()
new_cols = [cols[-1]] + cols[:-1]
player_of_the_week_df = player_of_the_week_df[new_cols]
player_of_the_week_df = player_of_the_week_df.rename(columns={'true_player_id': 'player_id'})
player_of_the_week_df.to_csv('player-of-the-week.csv')

# Path to your credentials file
credentials_path = 'cis-5450-final-project-485661e2f371.json'

# Set up the client with your credentials
storage_client = storage.Client.from_service_account_json(credentials_path)

# Specify your bucket name
bucket_name = 'nba_award_predictor'
bucket = storage_client.bucket(bucket_name)

# Define blob (file in GCS) and upload from the local file
blob = bucket.blob('nba_data/player-of-the-week.csv')
blob.cache_control = "max-age=0"
blob.upload_from_filename('player-of-the-week.csv')

print(f"File uploaded to gs://{bucket_name}/nba_data/player-of-the-week.csv")


File uploaded to gs://nba_award_predictor/nba_data/player-of-the-week.csv


In [21]:
player_of_the_week_df.player.nunique()

13

In [7]:
name_mapping_df.dtypes

in_table_name      object
nba_lookup_name    object
dtype: object

In [8]:
name_mapping_df

,in_table_name,nba_lookup_name
0,Jimmy Butler,Jimmy Butler III
1,J.D. Davison,JD Davison
2,Ha Ha,Ha Seung-jin
3,"Bear, The Body Hoffman",Paul 'The Bear' Hoffman
4,Kenyon Martin Jr.,KJ Martin
5,"Ted, Hound Dog McClain",Ted 'Hound Dog' McClain
6,Tre Scott,Trevon Scott
7,Sun Sun,Sun Yue
8,O.G. Anunoby,OG Anunoby
9,Brandon Boston Jr.,Brandon Boston


In [46]:
# # Bring in nba player lookup table to map the cleaned names to player IDs. Same player IDs from the NBA API.
# url = 'https://storage.googleapis.com/nba_award_predictor/nba_data/nba_player_lookup.csv'
# response = requests.get(url)
# if response.status_code == 200:
#     # Read in the name_mapping csv
#     nba_player_lookup_df = pd.read_csv('nba_player_lookup.csv')
# else:
#     print(f"Failed to retrieve data: Status code {response.status_code}")
#     print(response.text)  # Print the response content for debugging

query = """
SELECT * FROM nba_player_lookup_df
WHERE  1=1
--AND player_name LIKE '%Dog%'
AND player_id = '1630178'
"""

testing2 = duckdb.query(query).df()
testing2

,player_id,player_name,first_name,last_name,is_active
0,1630178,Tyrese Maxey,Tyrese,Maxey,True


In [30]:
query = """
SELECT player_name as nba_lookup_name, player_id
FROM nba_player_lookup_df
WHERE player_name IN(
'Jimmy Butler III',
'JD Davison',
'Ha Seung-jin',
--"Paul 'The Bear' Hoffman",
'KJ Martin',
--"Ted 'Hound Dog' McClain",
'Trevon Scott',
'Sun Yue',
'OG Anunoby',
'Brandon Boston',
'Chaundee Brown Jr.',
'Nigel Hayes-Davis',
'Ruben Nembhard Jr.',
'DJ Stewart',
'Yi Jianlian',
'Sun Yue',
'Reggie Bullock Jr.',
'Goga Bitadze',
'Ha Seung-jin',
'Matt Hurt',
'Ruben Nembhard Jr.',
'JR Smith',
'CJ McCollum',
'Lafayette Lever',
'Anfernee Hardaway',
'Billyray Bates',
'Jaren Jackson Jr.',
'Eddie Johnson',
'Nate Archibald',
'Bob Hansen',
'Steven Smith',
'Clifford Robinson',
'Dunleavy',
'World Free'
)
"""

testing2 = duckdb.query(query).df()
testing2

,nba_lookup_name,player_id
0,OG Anunoby,1628384
1,Nate Archibald,76054
2,Billyray Bates,76121
3,Goga Bitadze,1629048
4,Brandon Boston,1630527
5,Chaundee Brown Jr.,1630602
6,Reggie Bullock Jr.,203493
7,Jimmy Butler III,202710
8,JD Davison,1631120
9,World Free,76753
